In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data

In [2]:
df = pd.read_csv('data_20180119_20180420.csv', index_col=0)
df.head()

,t0,T1,K1,C1,T2,K2,C2,S0,Adj_S0
0,2018-01-02,2018-01-19,1000.0,191.525,2018-04-20,800.0,396.725000,59.4505,1189.01001
1,2018-01-02,2018-01-19,1100.0,92.525,2018-04-20,1015.0,191.675000,59.4505,1189.01001
2,2018-01-02,2018-01-19,1150.0,46.075,2018-04-20,1025.0,183.025000,59.4505,1189.01001
3,2018-01-02,2018-01-19,1160.0,38.000,2018-04-20,1050.0,157.776341,59.4505,1189.01001
4,2018-01-02,2018-01-19,1170.0,30.450,2018-04-20,1080.0,137.850000,59.4505,1189.01001


In [3]:
# S0 at different initial date
S0 = df['Adj_S0'].unique()

In [4]:
t0List = list(df.t0.unique())

# dimension: K[k-th t0][t1 and t2][i-th option with i-th strike]
K = [[np.array(df.loc[df.t0 == t0,'K1']),np.array(df.loc[df.t0==t0,'K2'])] for t0 in t0List]

# dimension: Pi[k-th t0][t1 and t2][i-th option with i-th price]
Pi = [[np.array(df.loc[df.t0 == t0,'C1']),np.array(df.loc[df.t0==t0,'C2'])] for t0 in t0List]

In [5]:
# number of t0
N_t0 = len(t0List)

# number of times (t1, t2)
T = 2

# number of options in hedging
N1, N2 = 20, 20
Size = max(N1, N2)

### Marginal Distribution: Uniform

In [6]:
# Assume marginal distribution is uniform
def unifMargDistr(eps, batch_size):
    # eps: 1/2 of the range of the simulated stock prices
    # batch_size: 1000
    data = np.random.uniform(size=(N_t0, batch_size, T))
    for i in range(N_t0):
        for t in range(T):
            data[i,:,t] = (S0[i]-eps) + 2*eps*data[i,:,t]
    return data

### Variables

In [13]:
def parameters(N_t0, T, Size, BATCH_SIZE):
    # parameter: d
    d = tf.Variable(
        initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
        trainable = True,
        name = 'd'
    )
    
    # parameter: lambda
    Lambda = tf.Variable(
        initial_value = tf.random.normal(shape=[N_t0, T, Size], dtype=tf.float64),
        trainable = True,
        name = 'lambda'
    )
    
    # parameter: Delta_0
    Delta_0 = tf.Variable(
        initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
        trainable = True,
        name = 'Delta0'
    )
    
    # neural network: Delta_1(S_1)
    Delta_1 = tf.keras.Sequential([
        tf.keras.layers.Dense(100, activation='relu', input_shape=(BATCH_SIZE,)),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(N_t0, activation='linear', dtype=tf.float64)
    ])
    
    return d, Lambda, Delta_0, Delta_1

### Optimisation

In [8]:
# Combine all the computation as a function
def ObjFunc(d, Lambda, Delta_0, Delta_1, K, Pi, Phi, S):
    hedgePrice = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
        hedgePrice.append(p)
    
    hedgeTerm = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*tf.maximum(S[n,:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
        p += Delta_0[n,0]*(S[n,:,0]-S0[n])
        p += Delta_1(S[n,:,0].reshape((1,-1)))[0,n]*(S[n,:,1]-S[n,:,0])
        hedgeTerm.append(p)
    
    Price = tf.reduce_mean(hedgePrice, axis=1)
    Gamma = 100
    diff = Phi-tf.stack(hedgeTerm, axis=0)
    Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)
    
    return Price + Gamma*Penalty, Price

In [18]:
def training():
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)
    nIter = 10000
    threshold = 1e-3
    prev_price = tf.ones((N_t0,), dtype = tf.float64) * float('inf')
    
    for step in range(nIter):
        if (step+1) % 500 == 0:
            print("Processing progress: %d / %d" %(step+1, nIter))
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(d)
            tape.watch(Lambda)
            tape.watch(Delta_0)
            for variable in Delta_1.trainable_variables:
                tape.watch(variable)
    
            obj, price = ObjFunc(d, Lambda, Delta_0, Delta_1, K, Pi, Phi, S)
        gradients_d, gradients_Lambda, gradients_Delta_0 = tape.gradient(obj, [d, Lambda, Delta_0])
        gradients_Delta_1 = tape.gradient(obj, Delta_1.trainable_variables)
        
        optimizer.apply_gradients(
            zip([gradients_d, gradients_Lambda, gradients_Delta_0] + gradients_Delta_1, 
                [d, Lambda, Delta_0] + Delta_1.trainable_variables)
        )
        
        diff = tf.abs(price - prev_price)
        if tf.reduce_max(diff) < threshold:
            print("Converged at step", step+1)
            break
        
        prev_price = price
        
        del tape
        
    return price, obj

### Optimisation with Simulated Stock Prices with Different Ranges 

In [19]:
BATCH_SIZE = 1000
epsList = [100, 200, 300, 400, 500]
Price = []
Obj = []
for eps in epsList:
    print("Maginal distribution of the stock price is uniform on [S0 - %d, S0 + %d]:" %(eps, eps))
    
    S = unifMargDistr(eps, BATCH_SIZE)
    Phi = (S[:,:,1]-S[:,:,0])
    
    d, Lambda, Delta_0, Delta_1 = parameters(N_t0, T, Size, BATCH_SIZE)
    price, obj = training()
    
    Price.append(price.numpy())
    Obj.append(obj.numpy())

Maginal distribution of the stock price is uniform on [S0 - 100, S0 + 100]:
Processing progress: 500 / 10000
Processing progress: 1000 / 10000
Processing progress: 1500 / 10000
Processing progress: 2000 / 10000
Processing progress: 2500 / 10000
Processing progress: 3000 / 10000
Processing progress: 3500 / 10000
Processing progress: 4000 / 10000
Processing progress: 4500 / 10000
Processing progress: 5000 / 10000
Processing progress: 5500 / 10000
Processing progress: 6000 / 10000
Processing progress: 6500 / 10000
Processing progress: 7000 / 10000
Processing progress: 7500 / 10000
Processing progress: 8000 / 10000
Processing progress: 8500 / 10000
Processing progress: 9000 / 10000
Processing progress: 9500 / 10000
Processing progress: 10000 / 10000
Maginal distribution of the stock price is uniform on [S0 - 200, S0 + 200]:
Processing progress: 500 / 10000
Processing progress: 1000 / 10000
Processing progress: 1500 / 10000
Processing progress: 2000 / 10000
Processing progress: 2500 / 10000

In [20]:
Price

[array([ -7067.75396309,  -4213.53005777,  -1982.85971257,  -3227.23857575,
         -4568.54740901,  -6121.78501826,  -4746.55761027,  -8582.5735219 ,
         -6688.79131826,  -7384.02525145, -12519.51149154, -14250.47971511,
         -8149.52341277]),
 array([ -291.66788073,   758.51769668,  2152.02272981,  1250.17312526,
         -293.57710233,     9.9151921 ,  2113.53295672,   382.27815129,
          349.07625902,    19.38590958, -2031.6934087 , -2482.49481356,
        -2959.16257797]),
 array([-372.95332658,  161.90139994,  178.42017094,  330.97699496,
         471.77623366,  613.13327677,  105.77903974,  351.76493409,
         879.99623752,  -99.44993565,    6.39424874, -946.98435583,
        -592.96188886]),
 array([1.50898457e+03, 8.02396275e+02, 5.16934633e+02, 3.15125552e+02,
        4.01473720e+02, 7.91430226e+02, 6.05000016e+02, 4.10049855e+02,
        1.22484815e+03, 2.69793753e+02, 2.50282799e+02, 2.63237501e+02,
        1.59455092e-01]),
 array([1508.88209546, 2859.6126